 # TASK 5- Stock_Market_Prediction_And_Forecasting_Using_Stacked_LSTM

# Dataset link: https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv

# Import necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import LSTM
%matplotlib inline

# Import the data and remove rows containing NAN values

In [ ]:
df = pd.read_csv('NSE-TATAGLOBAL.csv')
df=df. iloc[::-1]
df.head()

In [ ]:
!pip install pandas

In [ ]:
df.tail()

# Data Preprocessing

In [ ]:
df.isnull().sum()

# No null values found

In [ ]:
df.shape

In [ ]:
df_High=df.reset_index()['High']
plt.plot(df_High)
plt.title("High")

# Since LSTM are sensitive to the scale of the data, so we apply MinMax Scaler to transform our values between 0 and 1

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))
df_High = scaler.fit_transform(np.array(df_High).reshape(-1,1))

In [ ]:
df_High.shape

In [ ]:
df_High

# Split the data into train and test split

In [ ]:
training_size = int(len(df_High) * 0.75)
test_size = len(df_High) - training_size
train_data, test_data = df_High[0:training_size,:], df_High[training_size:len(df_High),:1]

In [ ]:
training_size,test_size

In [ ]:
def create_dataset(dataset, time_step = 1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
x_train, y_train = create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

# Reshape the input to be [samples, time steps, features] which is the requirement of LSTM

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [ ]:
print(x_train.shape), print(y_train.shape)

In [ ]:
print(x_test.shape), print(y_test.shape)

# Create the LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (100,1)))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100, batch_size = 64, verbose = 1)

# Lets predict and check performance metrics

In [ ]:
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)

# Transform back to original form

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calculate RMSE performance metrics

In [ ]:
math.sqrt(mean_squared_error(y_train, train_predict))

In [ ]:
!pip install y_train

# Test Data RMSE

In [ ]:
math.sqrt(mean_squared_error(y_test, test_predict))